In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!pip install nltk

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
from dateutil import parser
import missingno as msno
import re
from scipy.stats import mode
import spacy
import tqdm
import pickle
from bs4 import BeautifulSoup
from spacy.matcher import Matcher
import networkx as nx
import nltk
from nltk import TreebankWordTokenizer,WordPunctTokenizer
from nltk.tokenize import word_tokenize,PunktSentenceTokenizer

In [2]:
pd.set_option("display.max_column",500)
pd.set_option("display.max_rows",500)
sns.set(style="darkgrid")
sns.set_context("talk")

In [3]:
#load data
data=pd.read_csv(r"E:\Python\Msc-AI-Projects\loan.csv",low_memory=False)

In [ ]:
#Function to create meta data of the data
def create_Meta(data):
    meta_data={'dtypes':data.dtypes,
              'missing_values':round(data.isna().sum()*100/data.shape[0],3),
              'count':[data[i].count() for i in data.columns.values],
              'Number_Uniques':[data[i].nunique() for i in data.columns.values]}
    return pd.DataFrame(meta_data)

In [ ]:
Meta_Data=create_Meta(data)

In [ ]:
#Display meta data of our data
Meta_Data.T

In [ ]:
#Sorting Meta Data on the basis of missing value percentage in descending order
Meta_Data=Meta_Data.sort_values(by='missing_values',ascending=False)

In [ ]:
#Slicing Meta Data on basis of missing value percentage which shall be greater than 30%
Meta_Data[Meta_Data['missing_values']>30]

In [ ]:
#We have noticed that there are many columns which have more than 90 percent missing values
# There are columns which have 30% and 64% missing values also . 
#We will drop columns having missing values more than 50% and for rest we will explore and then take a call
missing_value_columns=Meta_Data[Meta_Data['missing_values']>50].index

In [ ]:
#Dropped missing value columns
ft_data=data.drop(missing_value_columns,axis=1)

In [ ]:
#Shape after and before dropping missing value columns
#We notice that 50% approx columns have been dropped
ft_data.shape,data.shape

In [ ]:
ft_data.head()

In [ ]:
def find_pattern(x):
    r=r"[0-9]"
    start,end=re.search(r,x).span()
    x=x[start:end+1]
    return x

In [ ]:
indx=ft_data[ft_data["emp_length"].notnull()]['emp_length'].index.values.tolist()
ft_data['emp_length'].loc[indx,]=ft_data['emp_length'].loc[indx,].map(find_pattern)

In [ ]:
#data cleaning
def data_split(a,x,y):
    b=a.split(x)[y]
    return b

ft_data['term']=ft_data.term.apply(data_split,args=(" ",1))


In [ ]:
ft_data["int_rate"]=ft_data.int_rate.apply(data_split,args=("%",0))

In [ ]:

# ft_data['emp_length'].loc[indx,]=ft_data['emp_length'].loc[indx,].apply(data_split,args=("+",0))
indx=ft_data[ft_data["revol_util"].notnull()]['revol_util'].index.values.tolist()
ft_data['revol_util'].loc[indx,]=ft_data['revol_util'].loc[indx,].apply(data_split,args=("%",0))

In [ ]:
#Checking if missing data is not at Random
#Creating dichotomous column for missing variable,1->missing,0->not missing
ft_data['desc_isMissing']=np.where(ft_data['desc'].isna(),1,0)
ft_data['emp_title_isMissing']=np.where(ft_data['emp_title'].isna(),1,0)
ft_data['emp_length_isMissing']=np.where(ft_data['emp_length'].isna(),1,0)

In [ ]:
ft_data.loan_status.value_counts()

In [ ]:
#We find that 80% of description of loan provided by borrower is missing for those who have fully paid 
ft_data[ft_data.desc_isMissing==1]['loan_status'].value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
#We find that 80% of job title provided by borrower is missing for those who have fully paid
ft_data[ft_data.emp_title_isMissing==1]['loan_status'].value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
#We find that 70% of job tenure provided by borrower is missing for those who have fully paid
ft_data[ft_data.emp_length_isMissing==1]['loan_status'].value_counts(normalize=True).plot(kind='bar')
plt.show()

It has been seen that majority of the missing data are of those who have fully paid, that is those who are likely to fully pay the loan amount. Hence we can say that the missing data is not at random. There is a clear pattern which we can see

In [ ]:
descMissing_data=ft_data[ft_data.desc_isMissing==1]

In [ ]:
descMissing_data.head()

In [ ]:
print(descMissing_data.loan_amnt.mean(),ft_data[ft_data.desc_isMissing==0]['loan_amnt'].mean(),ft_data['loan_amnt'].mean())

In [ ]:
print(descMissing_data.funded_amnt.mean(),ft_data[ft_data.desc_isMissing==0]['funded_amnt'].mean(),ft_data['funded_amnt'].mean())

In [ ]:
descMissing_data.term.value_counts()

In [ ]:
ft_data[ft_data.desc_isMissing==0]['term'].value_counts()

We see that distribution of term on the basis of description missing and not missing are similar that is loan applicants prefference for 3 years is twice more than of 5 years

In [ ]:
ft_data.int_rate=ft_data.int_rate.astype('float64')
descMissing_data.int_rate=descMissing_data.int_rate.astype('float64')
print(descMissing_data.int_rate.mean(),ft_data[ft_data.desc_isMissing==0].int_rate.mean(),ft_data.int_rate.mean())

In [ ]:
ft_data.emp_length=ft_data.emp_length.astype('float64')
descMissing_data.emp_length=descMissing_data.emp_length.astype('float64')
print(descMissing_data.emp_length.mean(),ft_data[ft_data.desc_isMissing==0].emp_length.mean(),ft_data.emp_length.mean())

In [ ]:
ft_data.revol_util=ft_data.revol_util.astype('float64')
descMissing_data.revol_util=descMissing_data.revol_util.astype('float64')
print(descMissing_data.revol_util.mean(),ft_data[ft_data.desc_isMissing==0].revol_util.mean(),ft_data.revol_util.mean())

No clear pattern in interest rate, revol util and employee length seen for missing description and non missing data 

In [ ]:
#Try Finding Employee Length based on Title and imputing the missings
title_wise_empLen=ft_data.groupby('emp_title',as_index=False,dropna=True)['emp_length'].median()

In [ ]:
emp_title_NA_Length=ft_data[(~ft_data.emp_title.isna())&(ft_data.emp_length.isna())]['emp_title'].values.tolist()

In [ ]:
a=pd.DataFrame()
for i in emp_title_NA_Length:
    a=pd.concat([a,title_wise_empLen[title_wise_empLen.emp_title==i]],axis=0)

In [ ]:
emp_title=a[~a.emp_length.isna()].emp_title.tolist()

In [ ]:
indexes=[]
for i in emp_title:
    ind=ft_data[(ft_data.emp_title==i)&(ft_data.emp_length.isna())]['emp_length'].index.values[0]
    indexes.append(ind)

In [ ]:
indexes

In [ ]:
missing_emp_length_data=ft_data.loc[indexes,:]
emp_lnt=a[~a.emp_length.isna()].emp_length.tolist()    
missing_emp_length_data.loc[:,'emp_length']=emp_lnt


In [ ]:
ft_data=pd.concat([ft_data.drop(indexes,axis=0),missing_emp_length_data],axis=0)

In [ ]:
ft_data.reset_index(drop=True,inplace=True)

In [ ]:
indexes=ft_data[(ft_data.emp_length.isna())].index.values.tolist()

In [ ]:
#Dropping rows where emp_length is NA
ft_data.drop(indexes,axis=0,inplace=True)

In [ ]:
ft_data.isna().sum()*100/ft_data.shape[0]

In [ ]:
#Grouping employee title most frequent based on employee experience
#Treating Missing Values of Employee title
empTitle_by_Length=ft_data.groupby("emp_length",as_index=False).agg({"emp_title":mode})
for i in range(len(empTitle_by_Length)):
    empTitle_by_Length.emp_title[i]=empTitle_by_Length.emp_title[i].mode[0]

In [ ]:
empTitle_by_Length

In [ ]:
indexes=ft_data[ft_data.emp_title.isna()].index.values.tolist()
empTitle_NAdata=ft_data.loc[indexes,:]
empTitle_NAdata[(empTitle_NAdata.emp_length==1.0)&(empTitle_NAdata.emp_title.isna())]["emp_title"].replace(np.nan,"Bank of America",inplace=True)

In [ ]:
empTitle_NAdata.emp_length.value_counts().sum()

In [ ]:
#Imputation of missing values in employee title based on employee length
a=pd.DataFrame()
for i in empTitle_NAdata.emp_length.unique().tolist():
    dummy=empTitle_NAdata[(empTitle_NAdata.emp_length==i)&(empTitle_NAdata.emp_title.isna())]
    dummy.emp_title.replace(np.nan,empTitle_by_Length[empTitle_by_Length.emp_length==i].emp_title.values[0],inplace=True)
    a=pd.concat([dummy,a],axis=0)
    
ft_data=pd.concat([ft_data.drop(indexes,axis=0),a],axis=0)

In [ ]:
ft_data.isna().sum()*100/ft_data.shape[0]

In [ ]:
ft_data['pub_rec_bankruptcies'].value_counts(dropna=False)

In [ ]:
ft_data.groupby('pub_rec_bankruptcies',dropna=False).agg({'loan_status':np.count_nonzero})

In [ ]:
#We notice the 697 missing public record of bankcruptcies are not the ones which are either charged off or who have fully paid..
#They belong to Current loan applicants
#Either this NaN can be another category itself or they belong to one specific category
ft_data[ft_data['pub_rec_bankruptcies'].isna()].groupby("loan_status",dropna=False)["pub_rec_bankruptcies"].count()

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average loan amount
#We see that those who have frequently shown bankcruptcies are the ones who have applied for high loan amounts. 
#The missing 697 are the ones which have applied for least loan amount
ft_data.groupby('pub_rec_bankruptcies',dropna=False).agg({'loan_amnt':'mean'})

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average funded amount
#There is difference in loan applied amount and funded amount
#We see that those who do not have bankcruptcies or who have maximum record of bankcruptcies that is 2 have the highest amount funded. 
#The missing 697 are the ones which have got lesser funded amount
ft_data.groupby('pub_rec_bankruptcies',dropna=False).agg({'funded_amnt':'mean'})

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average loan amount and loan status
#We see for all the number of bankcruptcies, people who have fully paid are the ones who have applied for the least amount and those who are current have the highest loan amount.
#For NAN category those whose loan is being charged off has the highest loan amount 
ft_data.groupby(['pub_rec_bankruptcies','loan_status'],dropna=False).agg({'loan_amnt':'mean'})

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average loan amount and loan status
#We see for all the number of bankcruptcies, people who have fully paid are in majority numbers and the current applicants are the least
#Of NaNs i.e 697 applicants 83% are already paid off and only 17% have been charged of
ft_data.groupby(['pub_rec_bankruptcies','loan_status'],dropna=False).agg({'loan_amnt':'count'})

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average interest rate
#We see that as the number of bankruptcies increases, it increases when the average interest rate rises. 
#The missing 697 have the least interest rate @11% approximately
ft_data.groupby('pub_rec_bankruptcies',dropna=False).agg({'int_rate':'mean'})

In [ ]:
#Understanding the behavior of bunkcruptcies on basis of average loan amount and installment amount
#We see for all the number of bankcruptcies, people who have public record of bankcruptcies as two, pays highest isntallment
#For NAN category the installment is lesser comparatively
ft_data.groupby(['pub_rec_bankruptcies'],dropna=False).agg({'installment':'mean'})

In [ ]:
#We see for zero and NaN category of the public record of bankcruptcies, majority of the applicants have home ownership as Rent and then Mortgage
#Those who displays bankcruptcies have higher frequencies of martgage as home ownership
ft_data.groupby(["pub_rec_bankruptcies","home_ownership"],dropna=False).agg({"emp_title":"count"})

In [ ]:
#Since the patterns seen above we see lot of similarities in NaN category and bankcruotcies class 0 , which is suggestive of that applicants who are having public_bankcrupticies as NaN have zero bankcruptcies
#Hence We can impute NaNs with 0
ft_data.pub_rec_bankruptcies.fillna(0.0,inplace=True)

In [ ]:
ft_data.isna().sum()*100/ft_data.shape[0]

In [ ]:
len(ft_data[~ft_data.desc.isna()].index)

In [ ]:
ft_data.desc[0]

In [ ]:
nlp=spacy.load("en_core_web_md")

In [ ]:
stop_words=nlp.Defaults.stop_words

In [ ]:
def remove_stopwords(text):
    if len(str(text))>0:
    
        nlp=spacy.load("en_core_web_md")
        stop_words=nlp.Defaults.stop_words
        text=[word for word in str(text).split(" ") if word.lower() not in stop_words]
        return " ".join([i for i in text])
    return " "

ft_data['cleaned_desc']=ft_data['desc'].map(remove_stopwords)

cleaned_desc=[]
for i in tqdm.tqdm(ft_data['desc']):
    cleaned_desc.append(remove_stopwords(i))

In [ ]:
ft_data['cleaned_desc']=ft_data['cleaned_desc'].map(remove_stopwords)

In [ ]:
ft_data['cleaned_desc'].head(20)

In [ ]:
ft_data.to_pickle(r"E:\Python\Msc-AI-Projects\ft_data.pkl")

In [4]:
ft_data=pd.read_pickle(r"E:\Python\Msc-AI-Projects\ft_data.pkl")

In [5]:
#Remove HTML tags
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [6]:
ft_data['cleaned_desc']=ft_data['cleaned_desc'].map(strip_html_tags)

In [7]:
ft_data['cleaned_desc']=ft_data['cleaned_desc'].map(lambda x: str(x).strip())

In [8]:
nlp=spacy.load("en_core_web_md")
def remove_special_characters(text):
    remove_digits=False
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [9]:
ft_data['cleaned_desc']=ft_data['cleaned_desc'].map(remove_special_characters)

In [27]:
def lemmatize_word(text):
    nlp=spacy.load("en_core_web_md")
    if len(text.split())>=2:
        text=text.lower()
        text=nlp(text)
        text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in tq(text)])
        
        return text
    return text

In [ ]:
ft_data['cleaned_desc']=ft_data['cleaned_desc'].map(lemmatize_word)

In [ ]:
[0]

In [ ]:
doc=nlp(ft_data['cleaned_desc'][0])
for tok in doc:
    print(tok.text," ----> ",tok.dep_," ----> ",tok.pos_)

In [ ]:
ft_data['cleaned_desc'][1]

In [ ]:
def get_entities(sent):
    
    ## chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""

    #############################################################

    for tok in nlp(sent):
        
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
          # check: token is a compound word or not
          if tok.dep_ == "compound":
            prefix = tok.text
            # if the previous word was also a 'compound' then add the current word to it
            if prv_tok_dep == "compound":
                prefix = prv_tok_text + " "+ tok.text

          # check: token is a modifier or not
          if tok.dep_.endswith("mod") == True:
            modifier = tok.text
            # if the previous word was also a 'compound' then add the current word to it
            if prv_tok_dep == "compound":
                modifier = prv_tok_text + " "+ tok.text

          ## chunk 3
          if tok.dep_.find("subj") == True:
            ent1 = modifier +" "+ prefix + " "+ tok.text
            prefix = ""
            modifier = ""
            prv_tok_dep = ""
            prv_tok_text = ""      

          ## chunk 4
          if tok.dep_.find("obj") == True:
            ent2 = modifier +" "+ prefix +" "+ tok.text

          ## chunk 5  
        # update variables
        prv_tok_dep = tok.dep_
        prv_tok_text = tok.text
      #############################################################
    return [ent1.strip(), ent2.strip()]


In [ ]:

entity_pairs = []

for i in tqdm.tqdm(ft_data['cleaned_desc']):
    entity_pairs.append(get_entities(i))

In [ ]:
def get_relation(sent):
    doc = nlp(sent)
    # Matcher class object 
    matcher = Matcher(nlp.vocab)
    #define the pattern 
    pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}]
    matcher.add("matching_1",[pattern]) 
    matches = matcher(doc)
    k = len(matches) 
    if k<1:
        return " "
    else:
        span = doc[matches[k][1]:matches[k][2]] 
        return(span.text)

In [ ]:
relations = [get_relation(i) for i in tqdm.tqdm(ft_data['cleaned_desc'])]

In [ ]:
relations

In [ ]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})

In [ ]:

# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=="added"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes
nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
doc=nlp(ft_data['cleaned_desc'][0])
for tok in doc:
    print(tok.text," ----> ",tok.dep_," ----> ",tok.pos_)

In [ ]:
ft_data.reset_index(drop=True,inplace=True)